In [1]:
## The input shape of the LSTM layer is 700 × 30, 700 × 60 or 700 × 90 
## where 700 is the amount of music
## The output of the LSTM layer is 700 × 128, 128 is the number of units in LSTM layer
## Additionally, the activation functions used in this project are the hyperbolic tangen
## and sigmoid

In [2]:
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from keras.layers import Bidirectional
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import regularizers
import pandas as pd
import sklearn as skl
import sklearn.model_selection as skm
from keras.optimizers import Adam
from keras.models import Model
import numpy as np
from keras import backend as K
import import_ipynb
import SVM_KNN as sknn

importing Jupyter notebook from SVM_KNN.ipynb
Accuracy on SVM for ZCR: 23.67%
Accuracy on SVM for MFCC: 64.33%
Accuracy on SVM for ZCR and MFCC: 62.00%
Accuracy on KNN for ZCR: 19.00%
Accuracy on KNN for MFCC: 31.67%
Accuracy on KNN for ZCR and MFCC: 31.67%


In [3]:
GTZAN_DIR = '/Users/nkkaushal/Documents/Fall22/ECS_271/Project/dataset/GTZAN_data/'
features = pd.read_csv(GTZAN_DIR+"features.csv")

In [4]:
def get_test_train_split(X, Y):

    scaler = skl.preprocessing.StandardScaler(copy=False)
    scaler.fit_transform(X)
    
    X_train, X_test, Y_train, Y_test = skm.train_test_split(X, Y, test_size=0.3, random_state=42)

    # Be sure training samples are shuffled.
    X_train, Y_train = skl.utils.shuffle(X_train, Y_train, random_state=42)
    X_test, Y_test = skl.utils.shuffle(X_test, Y_test, random_state=42)

    return [X_train, X_test, Y_train, Y_test]

In [5]:
X_ZCR = features[['ZRC' + str(ind) for ind in range(1,31)]]
X_MFCC = features[['MFCC' + str(ind) for ind in range(1,61)]]
X_ZCR_MFCC = features[['ZRC' + str(ind) for ind in range(1,31)]+['MFCC' + str(ind) for ind in range(1,61)]]
Y = features['label']

genre_list = Y.unique().tolist()

In [6]:
def one_hot(Y_genre_strings):
    y_one_hot = np.zeros((Y_genre_strings.shape[0], len(genre_list)))
    for i, genre_string in enumerate(Y_genre_strings):
        index = genre_list.index(genre_string)
        y_one_hot[i, index] = 1
    return y_one_hot

In [35]:
Y_one_hot = one_hot(Y)
X_train_zcr, X_test_zcr, Y_train_zcr, Y_test_zcr = get_test_train_split(X_ZCR, Y_one_hot)
X_train_zcr = X_train_zcr.to_numpy().reshape((X_train_zcr.shape[0], X_train_zcr.shape[1], 1))
X_test_zcr = X_test_zcr.to_numpy().reshape((X_test_zcr.shape[0], X_test_zcr.shape[1], 1))
# Y_train_zcr = Y_train_zcr.reshape((Y_train_zcr.shape[0], Y_train_zcr.shape[1], 1))
input_shape = (X_train_zcr.shape[1], X_train_zcr.shape[2])

In [7]:
# Y_one_hot = one_hot(Y)
X_train_mfcc, X_test_mfcc, Y_train_mfcc, Y_test_mfcc = get_test_train_split(X_MFCC, Y)
Y_one_hot_train = one_hot(Y_train_mfcc)
Y_one_hot_test = one_hot(Y_test_mfcc)
X_train_mfcc = X_train_mfcc.to_numpy().reshape((X_train_mfcc.shape[0], X_train_mfcc.shape[1], 1))
X_test_mfcc = X_test_mfcc.to_numpy().reshape((X_test_mfcc.shape[0], X_test_mfcc.shape[1], 1))
# Y_train_zcr = Y_train_zcr.reshape((Y_train_zcr.shape[0], Y_train_zcr.shape[1], 1))
input_shape = (X_train_mfcc.shape[1], X_train_mfcc.shape[2])

In [274]:
Y_one_hot = one_hot(Y)
X_train_zcr_mfcc, X_test_zcr_mfcc, Y_train_zcr_mfcc, Y_test_zcr_mfcc = get_test_train_split(X_ZCR_MFCC, Y_one_hot)
X_train_zcr_mfcc = X_train_zcr_mfcc.to_numpy().reshape((X_train_zcr_mfcc.shape[0], 1, X_train_zcr_mfcc.shape[1]))
X_test_zcr_mfcc = X_test_zcr_mfcc.to_numpy().reshape((X_test_zcr_mfcc.shape[0], 1, X_test_zcr_mfcc.shape[1]))
# Y_train_zcr = Y_train_zcr.reshape((Y_train_zcr.shape[0], Y_train_zcr.shape[1], 1))
input_shape = (X_train_zcr_mfcc.shape[1], X_train_zcr_mfcc.shape[2])

In [110]:
Y_train_zcr.shape

(700, 10)

In [8]:
inputs = Input(input_shape)
# lstm1 = LSTM(units=64, return_sequences=True)(inputs)
lstm = LSTM(units=128, return_sequences=False, dropout=0.05, recurrent_dropout=0.35)(inputs)
# lstm1 = LSTM(units=128, activation = 'relu', return_sequences=False)(lstm)
dense = Dense(units=10, activation="softmax")(lstm)

model = Model(inputs, dense)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-09 20:08:55.202799: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-09 20:08:55.203536: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 67,850
Trainable params: 67,850
Non-trainable params: 0
_________________________________________________________________


In [10]:
print("Training ...")
batch_size = 45 # num of training examples per minibatch
num_epochs = 500
model.fit(
    X_train_mfcc,
    Y_one_hot_train,
    batch_size=batch_size,
    epochs=num_epochs,
)

Training ...
Epoch 1/500


2022-11-09 20:08:57.993020: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-09 20:08:58.819545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 101s 6s/step - loss: 2.2271 - accuracy: 0.1786
Epoch 2/500
16/16 [==============================] - 94s 6s/step - loss: 2.1028 - accuracy: 0.2229
Epoch 3/500
16/16 [==============================] - 91s 6s/step - loss: 2.0050 - accuracy: 0.2529
Epoch 4/500
16/16 [==============================] - 93s 6s/step - loss: 1.9888 - accuracy: 0.2671
Epoch 5/500
16/16 [==============================] - 96s 6s/step - loss: 1.9351 - accuracy: 0.2929
Epoch 6/500
16/16 [==============================] - 95s 6s/step - loss: 1.8735 - accuracy: 0.3214
Epoch 7/500
16/16 [==============================] - 96s 6s/step - loss: 1.8536 - accuracy: 0.3429
Epoch 8/500
16/16 [==============================] - 95s 6s/step - loss: 1.8198 - accuracy: 0.3471
Epoch 9/500
16/16 [==============================] - 92s 6s/step - loss: 1.7743 - accuracy: 0.3829
Epoch 10/500
16/16 [==============================] - 94s 6s/step - loss: 1.7575 - accuracy: 0.3600
Epoch 11/500
16/16 [

In [11]:
intermediate_layer_model = Model(inputs=model.input,
                                outputs=model.layers[1].output)


In [12]:
intermediate_output_train = intermediate_layer_model(X_train_mfcc)
intermediate_output_test = intermediate_layer_model(X_test_mfcc)

In [13]:
intermediate_output_train

<tf.Tensor: shape=(700, 128), dtype=float32, numpy=
array([[ 0.14445743,  0.14315665, -0.30218437, ..., -0.00953268,
        -0.14834774,  0.33777028],
       [ 0.1030496 , -0.42257425, -0.6499769 , ..., -0.02278215,
        -0.1345078 , -0.5614194 ],
       [ 0.01077301,  0.24146117, -0.85154617, ..., -0.00997227,
        -0.338966  , -0.27942055],
       ...,
       [-0.13700236, -0.21270564, -0.19340311, ..., -0.29518175,
        -0.12090628,  0.46958858],
       [ 0.05540041, -0.97227883,  0.98278826, ...,  0.26049358,
         0.09674841,  0.4492931 ],
       [-0.78443   , -0.6579025 ,  0.2721703 , ..., -0.00291448,
        -0.24397391, -0.88313586]], dtype=float32)>

In [14]:
sknn.SVM_score(intermediate_output_train, Y_train_mfcc, intermediate_output_test, Y_test_mfcc)

0.5266666666666666

In [15]:
model.save('LSTM_model')

INFO:tensorflow:Assets written to: LSTM_model/assets


In [148]:
test_loss, test_acc = model.evaluate(X_test_zcr, Y_test_zcr, verbose=2)

2022-11-03 22:04:54.423506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-03 22:04:54.478340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 - 0s - loss: 1.0753 - accuracy: 0.1767 - 447ms/epoch - 45ms/step


In [21]:
test_loss, test_acc = model.evaluate(X_test_mfcc, Y_one_hot_test, verbose=2)

2022-11-09 14:31:26.849565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 - 4s - loss: 1.5619 - accuracy: 0.4600 - 4s/epoch - 383ms/step


In [ ]:
test_loss, test_acc = model.evaluate(X_test_zcr_mfcc, Y_test_zcr_mfcc, verbose=2)

2022-11-04 03:45:55.908381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-04 03:45:56.003917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-04 03:45:56.101725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 - 1s - loss: 0.8142 - accuracy: 0.5967 - 943ms/epoch - 94ms/step


: 